In [1]:
from os import environ
from swiftclient import Connection
from swiftclient.multithreading import MultiThreadingManager
from queue import Queue
from swiftclient.exceptions import ClientException
from concurrent.futures import as_completed
import os
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



In [73]:
def get_conn(options):
    """
    Return a connection building it from the options.
    """
    return Connection(options['authurl'],
                      options['user'],
                      options['key'],
                      auth_version=options['auth_version'],
                      os_options=options['os_options'])

def get_from_queue(q, timeout=864000):
    while True:
        try:
            item = q.get(timeout=timeout)
            return item
        except QueueEmpty:
            # Do nothing here, we only have a timeout to allow interruption
            pass
        
def download_file(conn, obj, path):
    resp_headers, obj_contents = conn.get_object('BUDGET', obj)
    if obj.split("/")[0] in list_dir:
        os.chdir(path+obj.split("/")[0])
        with open(obj.split("/")[2], 'wb') as local:
            local.write(obj_contents)
    else:
        os.mkdir(path+obj.split("/")[0])
        list_dir.append(obj.split("/")[0])
        os.chdir(path+obj.split("/")[0])
        with open(obj.split("/")[2], 'wb') as local:
            local.write(obj_contents)
            
def download_file_in_same_folder(conn, obj, path):
    os.chdir(path)
    resp_headers, obj_contents = conn.get_object('BUDGET', obj)
    with open(obj.split("/")[1]+"."+obj.split(".")[1]+"."+obj.split(".")[2], 'wb') as local:
        local.write(obj_contents)
   
def fn(conn):
    pass
    #return print(conn.get_object('BUDGET', selected[0]))

def interruptable_as_completed(fs, timeout=86400):
    while True:
        try:
            for f in as_completed(fs, timeout=timeout):
                fs.remove(f)
                yield f
            return
        except TimeoutError:
            # Do nothing here, we only have a timeout to allow interruption
            pass
        
def create_connection():
    return Connection(os_options={
    "object_storage_url" : "https://object-store.api.pi.dsic.minint.fr/v1/AUTH_4b223388a74b4390924063c8b07d4caf/"},
    preauthtoken = "86ec7cf074094f1eb20f6ada6c5ab532",
                           insecure=True)


In [ ]:
conn = create_connection()

resp_headers, containers = conn.get_account()
info, objects = conn.get_container('BUDGET', full_listing=True)
selected = [i["name"] for i in objects]
path = "/home/admin/projet/dataprep-ab/download/data_source/"
len(selected)


In [3]:
path = "/home/admin/projet/dataprep-ab/download/data_source/"
list_dir = os.listdir(path)
list_dir

['773645.xml.gz',
 '689503.xml.gz',
 '684944.xml.gz',
 '672347.xml.gz',
 '777371.xml.gz',
 '623543.xml.gz',
 '775775.xml.gz',
 '786919.xml.gz',
 '687250.xml.gz',
 '751177.xml.gz',
 '614677.xml.gz',
 '840202.xml.gz',
 '648480.xml.gz',
 '822683.xml.gz',
 '691498.xml.gz',
 '697989.xml.gz',
 '657175.xml.gz',
 '799758.xml.gz',
 '821029.xml.gz',
 '733356.xml.gz',
 '787763.xml.gz',
 '727598.xml.gz',
 '758304.xml.gz',
 '716227.xml.gz',
 '714061.xml.gz',
 '655485.xml.gz',
 '696109.xml.gz',
 '615128.xml.gz',
 '775180.xml.gz',
 '675374.xml.gz',
 '702119.xml.gz',
 '697374.xml.gz',
 '641090.xml.gz',
 '682043.xml.gz',
 '647142.xml.gz',
 '678891.xml.gz',
 '689119.xml.gz',
 '636950.xml.gz',
 '621869.xml.gz',
 '759479.xml.gz',
 '831477.xml.gz',
 '728828.xml.gz',
 '626740.xml.gz',
 '786942.xml.gz',
 '827866.xml.gz',
 '724006.xml.gz',
 '721046.xml.gz',
 '727772.xml.gz',
 '733542.xml.gz',
 '721593.xml.gz',
 '799537.xml.gz',
 '665672.xml.gz',
 '686452.xml.gz',
 '659719.xml.gz',
 '696274.xml.gz',
 '613189.x

In [72]:
selected[1].split("/")
selected[1].split(".")


['20200801/612009/1810878', 'xml', 'gz']

In [60]:
selected = selected[0:50]


In [67]:
def main():
    thread_manager = MultiThreadingManager(
                create_connection,
                segment_threads=20,
                object_dd_threads=20,
                object_uu_threads=20,
                container_threads=20
            )
    conn=create_connection()
    o_downs = [thread_manager.object_dd_pool.submit(
                        #fn
                        download_file_in_same_folder, obj, path
                    ) for obj in selected]

    for o_down in interruptable_as_completed(o_downs):
        yield o_down.result()
        

In [ ]:
while True:
    next(main())

In [4]:
import glob
len(glob.glob(path+"/**"))

227623

In [87]:
len(selected)


50